# Project Specification

In 2016 California voters aproved Proposition 64, legalizing recreative marijuana usage. More information about the Proposal can be found at:

<a href="https://ballotpedia.org/California_Proposition_64,_Marijuana_Legalization_(2016)">California Proposition 64, Marijuana Legalization (2016)</a>

Those opposing the Proposal presented five main objections:
- (1) It would double the number of highway deaths;
- (2) It would increase the cultivation or marijuana near schools and parks;
- (3) It would increase the black market and cartel activity;
- (4) It would harm poor communities with dependency problems due to inceased availability;
- (5) It would increase criminality over time, specially in the areas marijuana dispensaries.

The goal of this project is to analyze such claims through time series and geospacial analytical tools, to assess whether those predictions became true since legalization. The focus will be on marijuana related detensions and in the general crime rate near dispensaries. Claims 2, 4 and 5 will be assessed.



**Data Sources:**

- 1) Data regarding marijuana dispensaries in Los Angeles: Can be obtained through the Yelp API.
- 2) Data regarding Los Angeles crimes: Can be obtained through the <a href="https://data.lacity.org/">Los Angeles Open Data Portal</a>.
- 3) Data regarding arrest in Los Angeles: Can be obtained through the same portal.
- 4) Demographic data from Los Angeles schools: Are pubicly available on <a href="http://www.lausd.k12.ca.us/lausd/offices/bulletins/">Los Angeles School Information Branch</a>.

All datasets have been separatedly downloaded and will be loaded from my GitHub to ensure forward compatibility.

# Load Packages

In [1]:
#!pip install -q yelp

In [2]:
#!pip install -q yelpapi

In [3]:
# Disabling the multiple messages generated by the new versions from Pandas and Matplotlib
import sys
import warnings
import matplotlib.cbook
if not sys.warnoptions:
    warnings.simplefilter('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=matplotlib.cbook.mplDeprecation)

# Data manipulation imports
import re
import json
import time
import numpy as np
import pandas as pd
from yelp.client import Client
from yelpapi import YelpAPI

# File manipulation imports for Google Colab
#from google.colab import drive
#drive.mount('/content/drive')
#import os
#os.chdir("/content/drive/My Drive/Colab Notebooks/Marijuana_Crime")

In [4]:
!ls

Marijuana_Crime_1.ipynb
files


# Dataset 1

Obtaining from Yelp data about Los Angeles dispensaries

In [5]:
# Yelp API key
# Link to get my API key: https://www.yelp.com/developers/v3/manage_app?app_created=True
my_api = 'NDtxfhU-kG8n6LH8XQQH12tBu7XpkdDQzxNcSlp97FjeoJ1zk5woFqKyvE012TH-jhXw3U8HjxqYO1nSw54dx5gUcazFw7NbHe1pDi70Y07cqFppanNOo-gCmnvAXnYx'

In [6]:
# Connect to Yelp
client_access = Client(my_api)

In [7]:
# Function to format the file with the API queries
def format_file(file_path,
                logfile = './files/log_file.txt',
                file_description = None):
  
  # Applying regular expressiosn to clean the file name
  # Adjusting the file extension
    try:
        ext = re.search('(?<!^)(?<!\.)\.(?!\.)', file_path).start()
    except:
        raise NameError('File could not be found in this path.')

    # Adjusting the timestamp for the file name
    try:
        stamp = re.search('(?<!^)(?<!\.)[a-z]+_[a-z]+(?=\.)', file_path).start()
    except:
        raise NameError('File could not be found in this path.')

    # Format the file name adding the timestamp
    formatted_name = f'{file_path[:stamp]}{round(time.time())}_{file_path[stamp:]}' 

    # In case there is no file description, generate one
    if not file_description:
        file_description = f'File generated in: {time.asctime(time.gmtime(round(time.time())))}'

    # Opening the log file and storing the formatted data file and its description
    with open(logfile, 'a+') as f:
        f.write(f'{formatted_name}: {file_description}\n')

    # Returning the formatted data file and its description
    return formatted_name, file_description


In [8]:
# Connecting to the Yelp API and retrieving 1000 samples
# That is, 1000 registers from stores classified as 'dispensaries'
def yelp_search(category,
                location,
                offset_number = 0,
                n_samples = 1000):
  
    # API
    yelp_api = YelpAPI(my_api)

    # Register from the last result
    last_result = round(time.time())

    # List to store results
    results = []

    # Size
    size = 50

    # Initialize the loop count
    loops = 0

    # Initialize the runs count
    run = 1

    # Initialize offset
    offset_count = offset_number

    # Loop to retrieve data:
    while loops < n_samples:
    
        print(f'Initializing query {run}')

        # Query
        posts = yelp_api.search_query(categories = category,
                                  location = location,
                                  offset = offset_count,
                                  limit = size)
    
        # Business related posts
        results.extend(posts['businesses'])

        # Increment the loop count
        loops += size

        # Increment offset
        offset_count += size

        # Wait 3 seconds to run next query
        time.sleep(3)

        # Increment runs
        run += 1

    # Once the loop is finished, obtain the formatted file name and description
    formatted_name, file_description = format_file(file_path = f'./files/file_{category}.json')

    # Opening the formatted file and dumping the query results in json format
    with open(formatted_name, 'w+') as f:
        json.dump(results, f)

    print(f'\nQuery finished. Number of stores found: {len(results)} {category}.')

    global timestamp
    timestamp = round(time.time())

    return print(f'\nThe last timestamp was: {timestamp}.')

In [9]:
# Run query with business category and city
yelp_search('cannabisdispensaries', 'los angeles', n_samples = 1000)

Initializing query 1
Initializing query 2
Initializing query 3
Initializing query 4
Initializing query 5
Initializing query 6
Initializing query 7
Initializing query 8
Initializing query 9
Initializing query 10
Initializing query 11
Initializing query 12
Initializing query 13
Initializing query 14
Initializing query 15
Initializing query 16
Initializing query 17
Initializing query 18
Initializing query 19
Initializing query 20

Query finished. Number of stores found: 297 cannabisdispensaries.

The last timestamp was: 1589727546.


In [10]:
# Open the JSON file, read it, and generat the final list
with open(f'files/{timestamp}_file_cannabisdispensaries.json', 'r') as f:
    la_dispensaries = json.load(f)

In [11]:
# Checking
la_dispensaries[5]

{'id': 'aAagpdwqDYNfVtlH39Csbg',
 'alias': 'kushfly-los-angeles-5',
 'name': 'Kushfly',
 'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/7LaiBUBA_t2w2irxevaunQ/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/kushfly-los-angeles-5?adjust_creative=1vpy6OEt4U7reaEsVwesKw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=1vpy6OEt4U7reaEsVwesKw',
 'review_count': 229,
 'categories': [{'alias': 'cannabisdispensaries',
   'title': 'Cannabis Dispensaries'}],
 'rating': 4.0,
 'coordinates': {'latitude': 34.127617, 'longitude': -118.34671},
 'transactions': [],
 'price': '$$',
 'location': {'address1': '',
  'address2': None,
  'address3': None,
  'city': 'Los Angeles',
  'zip_code': '90068',
  'country': 'US',
  'state': 'CA',
  'display_address': ['Los Angeles, CA 90068']},
 'phone': '+18555710420',
 'display_phone': '(855) 571-0420',
 'distance': 7703.481319255708}

In [12]:
# Function to create a dataframe with the dispensaries data
def organize_data(stores_list, df_name = 'df_stores'):
    
    # Convert the list to a dataframe
    df_name = pd.DataFrame(stores_list)

    # List with the desired columns
    col_list = ['name',
                'is_closed',
                'url',
                'rating',
                'price',
                'review_count']

    # Filter the dataframe to have only the desired columns
    df_name = df_name[col_list]

    return df_name

In [13]:
# Apply the function to create the dataframe
df_stores = organize_data(la_dispensaries)

In [14]:
df_stores.shape

(297, 6)

In [15]:
df_stores.head()

,name,is_closed,url,rating,price,review_count
0,California Caregivers Alliance,False,https://www.yelp.com/biz/california-caregivers...,4.5,NaN,248
1,Herbarium,False,https://www.yelp.com/biz/herbarium-west-hollyw...,4.5,$$,230
2,MedMen Los Angeles - DTLA,False,https://www.yelp.com/biz/medmen-los-angeles-dt...,4.0,$$,303
3,Green Earth Collective,False,https://www.yelp.com/biz/green-earth-collectiv...,4.5,$$,183
4,HERB,False,https://www.yelp.com/biz/herb-los-angeles-3?ad...,4.5,$$,99


In [16]:
# Extracting latitudes
latitude_list = [la_dispensaries[i]['coordinates']['latitude'] for i in range(len(la_dispensaries))]

In [17]:
# Extracting longitudes
longitude_list = [la_dispensaries[i]['coordinates']['longitude'] for i in range(len(la_dispensaries))]

In [18]:
# Add geolocation to dataframe
df_stores['latitude'] = latitude_list
df_stores['longitude'] = longitude_list

In [19]:
df_stores.head()

,name,is_closed,url,rating,price,review_count,latitude,longitude
0,California Caregivers Alliance,False,https://www.yelp.com/biz/california-caregivers...,4.5,NaN,248,34.082350,-118.272037
1,Herbarium,False,https://www.yelp.com/biz/herbarium-west-hollyw...,4.5,$$,230,34.088530,-118.344600
2,MedMen Los Angeles - DTLA,False,https://www.yelp.com/biz/medmen-los-angeles-dt...,4.0,$$,303,34.044600,-118.254440
3,Green Earth Collective,False,https://www.yelp.com/biz/green-earth-collectiv...,4.5,$$,183,34.122470,-118.210670
4,HERB,False,https://www.yelp.com/biz/herb-los-angeles-3?ad...,4.5,$$,99,34.043499,-118.250206


In [20]:
# Create a location column with a tuple containing latitude and longitude
df_stores['location'] = list(zip(df_stores['latitude'], df_stores['longitude']))

In [21]:
df_stores.head()

,name,is_closed,url,rating,price,review_count,latitude,longitude,location
0,California Caregivers Alliance,False,https://www.yelp.com/biz/california-caregivers...,4.5,NaN,248,34.082350,-118.272037,"(34.08235, -118.272037)"
1,Herbarium,False,https://www.yelp.com/biz/herbarium-west-hollyw...,4.5,$$,230,34.088530,-118.344600,"(34.08853, -118.3446)"
2,MedMen Los Angeles - DTLA,False,https://www.yelp.com/biz/medmen-los-angeles-dt...,4.0,$$,303,34.044600,-118.254440,"(34.0446, -118.25444)"
3,Green Earth Collective,False,https://www.yelp.com/biz/green-earth-collectiv...,4.5,$$,183,34.122470,-118.210670,"(34.12247, -118.21067)"
4,HERB,False,https://www.yelp.com/biz/herb-los-angeles-3?ad...,4.5,$$,99,34.043499,-118.250206,"(34.0434989929199, -118.250205993652)"


In [22]:
df_stores.shape

(297, 9)

In [23]:
# Saving the dataset
df_stores.to_csv('files/df_stores.csv')

# Dataset 2

Obtaining data from Los Angeles crimes between 2010 and 2019. Dataset downloaded from LA Open Data Portal. Step-by-step to download it is: 
- 1) Access: https://data.lacity.org/
- 2) Search: "Crime Data"
- 3) Choose: "Crime Data from 2010 to 2019"
- 4) Select: "View Data" -> "Export" -> "Download" -> "CSV"

Data dictonary: https://data.lacity.org/A-Safe-City/Crime-Data-from-2010-to-2019/63jg-8b9z

I've uploaded the CSV to the ./files folder in this project's directory.

In [24]:
# Loading the CSV with crime data
df_crimes = pd.read_csv('files/Crime_Data_from_2010_to_2019.csv')

In [25]:
df_crimes.shape

(2114238, 28)

In [26]:
df_crimes.head(3)

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,1307355,02/20/2010 12:00:00 AM,02/20/2010 12:00:00 AM,1350,13,Newton,1385,2,900,VIOLATION OF COURT ORDER,...,AA,Adult Arrest,900.0,NaN,NaN,NaN,300 E GAGE AV,NaN,33.9825,-118.2695
1,11401303,09/13/2010 12:00:00 AM,09/12/2010 12:00:00 AM,45,14,Pacific,1485,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",...,IC,Invest Cont,740.0,NaN,NaN,NaN,SEPULVEDA BL,MANCHESTER AV,33.9599,-118.3962
2,70309629,08/09/2010 12:00:00 AM,08/09/2010 12:00:00 AM,1515,13,Newton,1324,2,946,OTHER MISCELLANEOUS CRIME,...,IC,Invest Cont,946.0,NaN,NaN,NaN,1300 E 21ST ST,NaN,34.0224,-118.2524


In [27]:
df_crimes.isnull().sum()

DR_NO                   0
Date Rptd               0
DATE OCC                0
TIME OCC                0
AREA                    0
AREA NAME               0
Rpt Dist No             0
Part 1-2                0
Crm Cd                  0
Crm Cd Desc             0
Mocodes            227959
Vict Age                0
Vict Sex           196652
Vict Descent       196699
Premis Cd              53
Premis Desc           187
Weapon Used Cd    1404139
Weapon Desc       1404140
Status                  3
Status Desc             0
Crm Cd 1               10
Crm Cd 2          1975051
Crm Cd 3          2110747
Crm Cd 4          2114134
LOCATION                0
Cross Street      1758896
LAT                     0
LON                     0
dtype: int64

In [28]:
# Substituting the spaces in the columns' names for underscores
# To simplify filtering/indexation
df_crimes.columns = [column.lower().replace(' ', '_') for column in df_crimes.columns]

In [29]:
# Dropping unnecessary columns
df_crimes.drop(labels = ['crm_cd_1', 
                         'crm_cd_2', 
                         'crm_cd_3', 
                         'crm_cd_4',
                         'premis_cd',
                         'premis_desc', 
                         'vict_descent', 
                         'vict_sex',
                         'status',
                         'dr_no', 
                         'area_',
                         'date_rptd',
                         'rpt_dist_no',
                         'crm_cd',
                         'part_1-2',
                         'mocodes',
                         'cross_street',
                         'weapon_used_cd',
                         'status_desc',
                         'time_occ',
                         'vict_age'],
               axis = 1,
               inplace = True)

In [30]:
# Keeping the weapon_desc column and filling NAs with 'unknown'
df_crimes.weapon_desc.fillna('unknown', inplace = True)

In [31]:
df_crimes.isnull().sum()

date_occ       0
area_name      0
crm_cd_desc    0
weapon_desc    0
location       0
lat            0
lon            0
dtype: int64

In [32]:
df_crimes.head(3)

,date_occ,area_name,crm_cd_desc,weapon_desc,location,lat,lon
0,02/20/2010 12:00:00 AM,Newton,VIOLATION OF COURT ORDER,unknown,300 E GAGE AV,33.9825,-118.2695
1,09/12/2010 12:00:00 AM,Pacific,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",unknown,SEPULVEDA BL,33.9599,-118.3962
2,08/09/2010 12:00:00 AM,Newton,OTHER MISCELLANEOUS CRIME,unknown,1300 E 21ST ST,34.0224,-118.2524


In [33]:
# Shape
df_crimes.shape

(2114238, 7)

In [34]:
# Saving the dataframe to a CSV file
df_crimes.to_csv('files/df_crimes.csv')

# Dataset 3

Obtaining data from Los Angeles arrests from 2010 onwards. Dataset downloaded from LA Open Data Portal. Step-by-step to download it is:

1) Access: https://data.lacity.org/
2) Search: "Arrests"
3) Choose: "Arrest Data from 2010 to Present"
4) Select: "View Data" -> "Export" -> "Download" -> "CSV"
Data dictonary: https://data.lacity.org/A-Safe-City/Arrest-Data-from-2010-to-Present/yru6-6re4

I've uploaded the CSV to the ./files folder in this project's directory.

In [35]:
# Loading the Arrests CSV
df_arrests = pd.read_csv('files/Arrest_Data_from_2010_to_Present.csv')

In [36]:
df_arrests.shape

(1350103, 17)

In [37]:
df_arrests.head(3)

,Report ID,Arrest Date,Time,Area ID,Area Name,Reporting District,Age,Sex Code,Descent Code,Charge Group Code,Charge Group Description,Arrest Type Code,Charge,Charge Description,Address,Cross Street,Location
0,200110044,03/27/2020,2125.0,1,Central,142,20,F,W,13.0,Prostitution/Allied,M,647(B)PC,PROSTITUTION,400 S FIGUEROA ST,NaN,"(34.0535, -118.256)"
1,200110045,04/01/2020,1800.0,1,Central,166,44,F,H,18.0,Drunkeness,M,41.27(C)LAM,DRINKING IN PUBLIC***,WINSTON ST,SAN PEDRO ST,"(34.0421, -118.2469)"
2,200110271,03/08/2020,1545.0,1,Central,166,61,M,B,18.0,Drunkeness,M,41.27(C)LAM,DRINKING IN PUBLIC***,6TH,SAN JULIAN,"(34.0428, -118.2461)"


In [38]:
df_arrests.isnull().sum()

Report ID                        0
Arrest Date                      0
Time                           198
Area ID                          0
Area Name                        0
Reporting District               0
Age                              0
Sex Code                         0
Descent Code                     0
Charge Group Code            91986
Charge Group Description     92528
Arrest Type Code                 1
Charge                           0
Charge Description           91775
Address                          0
Cross Street                583097
Location                         0
dtype: int64

In [39]:
df_arrests.dtypes

Report ID                     int64
Arrest Date                  object
Time                        float64
Area ID                       int64
Area Name                    object
Reporting District            int64
Age                           int64
Sex Code                     object
Descent Code                 object
Charge Group Code           float64
Charge Group Description     object
Arrest Type Code             object
Charge                       object
Charge Description           object
Address                      object
Cross Street                 object
Location                     object
dtype: object

In [40]:
# Converting the 'Arrest Date' column from String to Datetime, to facilitate manipulation
df_arrests['Arrest Date'] = pd.to_datetime(df_arrests['Arrest Date'])

In [41]:
# Using regular expressions to clean the 'Location' column and convert it to a list
df_arrests['Location'] = df_arrests['Location'].map(lambda x: re.sub('[(),°]', '', x)).str.split()

In [42]:
df_arrests.head(3)

,Report ID,Arrest Date,Time,Area ID,Area Name,Reporting District,Age,Sex Code,Descent Code,Charge Group Code,Charge Group Description,Arrest Type Code,Charge,Charge Description,Address,Cross Street,Location
0,200110044,2020-03-27,2125.0,1,Central,142,20,F,W,13.0,Prostitution/Allied,M,647(B)PC,PROSTITUTION,400 S FIGUEROA ST,NaN,"[34.0535, -118.256]"
1,200110045,2020-04-01,1800.0,1,Central,166,44,F,H,18.0,Drunkeness,M,41.27(C)LAM,DRINKING IN PUBLIC***,WINSTON ST,SAN PEDRO ST,"[34.0421, -118.2469]"
2,200110271,2020-03-08,1545.0,1,Central,166,61,M,B,18.0,Drunkeness,M,41.27(C)LAM,DRINKING IN PUBLIC***,6TH,SAN JULIAN,"[34.0428, -118.2461]"


In [43]:
# Extracting latitude and longitude
df_arrests['latitude'] = df_arrests['Location'].map(lambda x: x[0])
df_arrests['longitude'] = df_arrests['Location'].map(lambda x: x[1])

In [44]:
# Converting the geolocation to float
df_arrests['latitude'] = df_arrests['latitude'].map(lambda x: float(x))
df_arrests['longitude'] = df_arrests['longitude'].map(lambda x: float(x))

In [ ]:
# Converting the 'Charge Description' column to string and lower case
df_arrests['Charge Description'] = df_arrests['Charge Description'].map(lambda x: str(x))
df_arrests['Charge Description'] = df_arrests['Charge Description'].map(lambda x: x.lower())

In [ ]:
df_arrests.shape

In [ ]:
df_arrests.head(3)

In [ ]:
# Listing all arrest classifications
arrests_list = list(df_arrests['Charge Description'].value_counts().index.sort_values())

In [ ]:
# Setting to lower case to standardize
arrests_list = [x.lower() for x in arrests_list]

In [ ]:
# Checking all arrest classifications
arrests_list

In [ ]:
# Filtering all marijuana arrests
# Some descriptions were abbreviated to 'marij'
marijuana_arrests = [x for x in arrests_list if 'marij' in x]

In [ ]:
len(marijuana_arrests)

In [ ]:
marijuana_arrests

In [ ]:
# Creating a new df column to identify arrests which were marijuana related
df_arrests['marijuana_related'] = df_arrests['Charge Description'].map(lambda x: x if x in marijuana_arrests else np.NaN)

In [ ]:
df_arrests.head(3)

In [ ]:
# Counting non-NA values to see how many marijuana arrests there were
len(df_arrests[~df_arrests['marijuana_related'].isnull()])

In [ ]:
# Keeping only the marijuana related data
df_arrests = df_arrests[~df_arrests['marijuana_related'].isnull()]

In [ ]:
df_arrests.shape

In [ ]:
df_arrests.head(3)

In [ ]:
# Saving dataframe as csv
df_arrests.to_csv('files/df_arrests.csv')

# Dataset 4

Obtaining geolocation data on Los Angeles schools.

Data sourced from: http://www.lausd.k12.ca.us/lausd/offices/bulletins/

File used: http://www.lausd.k12.ca.us/lausd/offices/bulletins/lausdk12.tab

In [ ]:
# Loading file
df_schools = pd.read_csv('files/lausdk12.tab', sep = '\t')

In [ ]:
df_schools.shape

In [ ]:
df_schools.head(3)

In [ ]:
# Merging Address + City + State + ZIP to create a new 'complete_address' column
df_schools['complete_address'] = df_schools['Address'] + ' ' + df_schools['City'] + ' ' + df_schools['State'] + ' ' + df_schools['Zip Code'].astype(str)

In [ ]:
df_schools['complete_address'] = df_schools['complete_address'].astype(str)

In [ ]:
# Deleting unnecessary columns
df_schools = df_schools.drop(['Address',
                             'City',
                             'State',
                             'Cost Center Code',
                             'Legacy Code',
                             'Telephone',
                             'Fax',
                             'Calendar',
                             'File Build Date'],
                            1)

In [ ]:
# Checking for duplicates
# This can happen if there are two schools registered on the same address, such as a kindergarten and middle school
df_schools = df_schools[~df_schools.duplicated(subset = 'complete_address')].sort_values('complete_address')

In [ ]:
# Resetting index
df_schools.reset_index(drop = True, inplace = True)

In [ ]:
df_schools.shape

In [ ]:
df_schools.head(3)

In [ ]:
# Saving the schools df
df_schools.to_csv('files/df_schools.csv')